In [ ]:
MySQL Config Editor 
=============================

-- helps in storing encrypted database credentials.
-- with filname .mylogin.cnf in users home directory.
-- default group is [client] and for any other group --login-path has to be specicified.


Set with default group 
===============================

[mysqluser@mysqlserver ~]$ mysql_config_editor set -u root -p
Enter password: 

[mysqluser@mysqlserver ~]$ mysql_config_editor print
[client]
user = "root"
password = *****

[mysqluser@mysqlserver ~]$ pwd
/home/mysqluser

[mysqluser@mysqlserver ~]$ ls -artl .mylogin.cnf
-rw-------. 1 mysqluser mysqluser 100 May 16 00:21 .mylogin.cnf
[mysqluser@mysqlserver ~]$ 


[mysqluser@mysqlserver ~]$ mysql

Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 8

mysql> select user();
+----------------+
| user()         |
+----------------+
| root@localhost |
+----------------+
1 row in set (0.00 sec)



Set with dbadmin_client group 
===========================================

[mysqluser@mysqlserver ~]$ mysql_config_editor set --login-path=dbadmin_client -u db_admin -p
Enter password: 
[mysqluser@mysqlserver ~]$ 


[mysqluser@mysqlserver ~]$ mysql_config_editor print
[client]
user = "root"
password = *****


[mysqluser@mysqlserver ~]$ mysql_config_editor print --all
[client]
user = "root"
password = *****
[dbadmin_client]
user = "db_admin"
password = *****


[mysqluser@mysqlserver ~]$ mysql --login-path=dbadmin_client

Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 9

mysql> select user();
+--------------------+
| user()             |
+--------------------+
| db_admin@localhost |
+--------------------+
1 row in set (0.00 sec)



Remove/Reset all the stored credentials
==================================================

[mysqluser@mysqlserver ~]$ mysql_config_editor reset
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysql_config_editor print --all
[mysqluser@mysqlserver ~]$ 

In [ ]:
Mysqladmin Program
===================================

-- is a program to carry out admin tasks like shutdown , create/drop database , check status , ping , 
   start/stop replica.


[mysqluser@mysqlserver ~]$ mysqladmin ping
mysqld is alive

[mysqluser@mysqlserver ~]$ mysqladmin status
Uptime: 1621  Threads: 2  Questions: 10  Slow queries: 0  Opens: 119  Flush tables: 3  Open tables: 38 
 Queries per second avg: 0.006

[mysqluser@mysqlserver ~]$ mysqladmin flush-tables 
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysqladmin status
Uptime: 1668  Threads: 2  Questions: 14  Slow queries: 0  Opens: 119  Flush tables: 4  Open tables: 0  
Queries per second avg: 0.008

[mysqluser@mysqlserver ~]$ mysqladmin shutdown 
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysqladmin ping
mysqladmin: connect to server at 'localhost' failed
Check that mysqld is running and that the socket: '/var/lib/mysql/mysql.sock' exists!

[mysqluser@mysqlserver ~]$ systemctl start mysqld
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysqladmin ping
mysqld is alive

[mysqluser@mysqlserver ~]$ mysqladmin create employeesdb
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysql
Welcome to the MySQL monitor.  Commands end with ; or \g.

mysql> show databases like '%emp%';
+------------------+
| Database (%emp%) |
+------------------+
| employeesdb      |
+------------------+

[mysqluser@mysqlserver ~]$ mysqladmin drop employeesdb

Do you really want to drop the 'employeesdb' database [y/N] y

Database "employeesdb" dropped

[mysqluser@mysqlserver ~]$ 


[mysqluser@mysqlserver ~]$ mysql
Welcome to the MySQL monitor.  Commands end with ; or \g.

mysql> show databases like '%emp%';
Empty set (0.00 sec)

In [ ]:
MySQL Executing SQL Files 
==========================================


using source or \. 
=============================

[mysqluser@mysqlserver ~]$ cat create_table_staff.sql 
use employeesdb;
create table staff (
	staff_id int primary key,
	staff_name varchar(50)
	staff_email varchar(100)
);
[mysqluser@mysqlserver ~]$ 


mysql> source create_table_staff.sql
Database changed
Query OK, 0 rows affected (0.06 sec)

mysql> show tables;
+-----------------------+
| Tables_in_employeesdb |
+-----------------------+
| staff                 |
+-----------------------+

mysql> drop table staff;
Query OK, 0 rows affected (0.01 sec)

mysql> \. create_table_staff.sql
Database changed
Query OK, 0 rows affected (0.03 sec)

mysql> show tables;
+-----------------------+
| Tables_in_employeesdb |
+-----------------------+
| staff                 |
+-----------------------+



using < operator
==================================

[mysqluser@mysqlserver ~]$ mysql < create_table_staff.sql 
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysql

Welcome to the MySQL monitor.  Commands end with ; or \g.

mysql> use employeesdb;

Database changed

mysql> show tables;
+-----------------------+
| Tables_in_employeesdb |
+-----------------------+
| staff                 |
+-----------------------+
1 row in set (0.00 sec)


using bash script 
======================================

[mysqluser@mysqlserver ~]$ vi create_table_staff.sh
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ cat create_table_staff.sh
mysql < $1
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ chmod 755 create_table_staff.sh
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ ./create_table_staff.sh create_table_staff.sql &
[1] 4661
[mysqluser@mysqlserver ~]$ 
[1]+  Done                    ./create_table_staff.sh create_table_staff.sql
[mysqluser@mysqlserver ~]$ 


pipe method 
=========================

[mysqluser@mysqlserver ~]$ cat create_table_staff.sql | mysql
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysql
Welcome to the MySQL monitor.  Commands end with ; or \g.

mysql> use employeesdb;

Database changed

mysql> show tables;
+-----------------------+
| Tables_in_employeesdb |
+-----------------------+
| staff                 |
+-----------------------+
1 row in set (0.00 sec)

In [ ]:
mysqlimport program
===================================

-- is a data import program , takes .txt file as input with tab delimited and table name as filename.
-- can be loaded only with location of secure_file_priv variable.

[mysqluser@mysqlserver ~]$ vi staff_data.txt
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ cat staff_data.txt 
1001 "Arvind" "gsddd@dfff.com"
1002 "Aarna" "hbchbcc@vfff.com"
1003 "Meghana" "xcxgsgfd@gdhad.com"
[mysqluser@mysqlserver ~]$ 


[mysqluser@mysqlserver ~]$ mysqlimport employeesdb staff_data.txt 

mysqlimport: Error: 1146, Table 'employeesdb.staff_data' doesn't exist, when using table: staff_data

[mysqluser@mysqlserver ~]$ 


[mysqluser@mysqlserver ~]$ mv staff_data.txt staff.txt
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysqlimport employeesdb staff.txt

mysqlimport: Error: 1290, The MySQL server is running with the --secure-file-priv option so it cannot 
execute this statement, when using table: staff


mysql> show variables like '%secure%';
+--------------------------+-----------------------+
| Variable_name            | Value                 |
+--------------------------+-----------------------+
| require_secure_transport | OFF                   |
| secure_file_priv         | /var/lib/mysql-files/ |
+--------------------------+-----------------------+
2 rows in set (0.00 sec)

[root@mysqlserver mysqluser]# mv staff.txt /var/lib/mysql-files/

[root@mysqlserver mysql-files]# chown mysql:mysql staff.txt 
[root@mysqlserver mysql-files]# 

[mysqluser@mysqlserver ~]$ mysqlimport employeesdb /var/lib/mysql-files/staff.txt
mysqlimport: Error: 1265, Data truncated for column 'staff_id' at row 1, when using table: staff
[mysqluser@mysqlserver ~]$ 

proper tab limited text file 
================================

[root@mysqlserver mysql-files]# vi staff.txt

[root@mysqlserver mysql-files]# cat staff.txt 
1001	Arvind Kamath	gdgfgff@fdff.com
1002	Aarna Kamath	fsdfsgdsdd@vafff.com
1003	Meghan Kamath	vxcxcxcv@hgfdad.com
[root@mysqlserver mysql-files]# 


[root@mysqlserver mysql-files]# mysqlimport -u root -p employeesdb /var/lib/mysql-files/staff.txt
Enter password: 
employeesdb.staff: Records: 3  Deleted: 0  Skipped: 0  Warnings: 0
[root@mysqlserver mysql-files]# 

mysql> select * from staff;
+----------+---------------+----------------------+
| staff_id | staff_name    | staff_email          |
+----------+---------------+----------------------+
|     1001 | Arvind Kamath | gdgfgff@fdff.com     |
|     1002 | Aarna Kamath  | fsdfsgdsdd@vafff.com |
|     1003 | Meghan Kamath | vxcxcxcv@hgfdad.com  |
+----------+---------------+----------------------+
3 rows in set (0.00 sec)



mysql> create table employees (
    -> emp_id int auto_increment primary key,
    -> emp_name varchar(50),
    -> emp_phone varchar(50),
    -> emp_email varchar(50)
    -> );
Query OK, 0 rows affected (0.04 sec)

[root@mysqlserver mysql-files]# mysqlimport -u root -p employeesdb /var/lib/mysql-files/employees.txt 
Enter password: 
mysqlimport: Error: 1366, Incorrect integer value: 'Ranganna' for column 'emp_id' at row 1, when using table: employees
[root@mysqlserver mysql-files]# 

[root@mysqlserver mysql-files]# mysqlimport -u root -p -c emp_name,emp_phone,emp_email employeesdb /var/lib/mysql-files/employees.txt
Enter password: 
employeesdb.employees: Records: 3  Deleted: 0  Skipped: 0  Warnings: 0
[root@mysqlserver mysql-files]# 

In [ ]:
mysqlcheck program
=======================================

-- is a table maintenence program , it checks , repairs , oprtimize or analyse the table.
-- table will be locked during the opertion.
-- -c check , -a analyze , -o optimize , -r repair.
-- -c is default.

[mysqluser@mysqlserver ~]$ mysqlcheck employeesdb employees
employeesdb.employees                              OK
[mysqluser@mysqlserver ~]$ 
                          
[mysqluser@mysqlserver ~]$ mysqlcheck -o -a employeesdb employees
Error:  mysqlcheck doesn't support multiple contradicting commands.
[mysqluser@mysqlserver ~]$ 

[mysqluser@mysqlserver ~]$ mysqlcheck -o employeesdb employees
employeesdb.employees
note     : Table does not support optimize, doing recreate + analyze instead
status   : OK

[mysqluser@mysqlserver ~]$ mysqlcheck -r employeesdb employees
employeesdb.employees
note     : The storage engine for the table doesn't support repair

[mysqluser@mysqlserver ~]$ mysqlcheck -a employeesdb employees
employeesdb.employees                              OK


simulate data corruption by editing the os table file
==========================================================

[root@mysqlserver employeesdb]# vi employees.ibd 
[root@mysqlserver employeesdb]# 
[root@mysqlserver employeesdb]# ls -rtl
total 0
-rw-r-----. 1 mysql mysql 0 May 16 19:11 employees.ibd
[root@mysqlserver employeesdb]# 

[mysqluser@mysqlserver ~]$ mysqlcheck -v -c employeesdb employees
# Connecting to localhost...
employeesdb.employees                              OK
# Disconnecting from localhost...

[root@mysqlserver employeesdb]# ls -rtl
total 0
-rw-r-----. 1 mysql mysql 0 May 16 19:11 employees.ibd
[root@mysqlserver employeesdb]# 

[root@mysqlserver employeesdb]# mysqlcheck -u root -p -v -r employeesdb employees
# Connecting to localhost...
Enter password: 
employeesdb.employees
note     : The storage engine for the table doesn't support repair
# Disconnecting from localhost...
[root@mysqlserver employeesdb]# 

[root@mysqlserver employeesdb]# mysqlcheck -u root -p -o employeesdb employees
Enter password: 
employeesdb.employees
note     : Table does not support optimize, doing recreate + analyze instead
status   : OK
[root@mysqlserver employeesdb]# 
[root@mysqlserver employeesdb]# ls -rtl
total 112
-rw-r-----. 1 mysql mysql 114688 May 16 19:17 employees.ibd
[root@mysqlserver employeesdb]# 

mysql> select * from employeesdb.employees;
+--------+----------+-----------+----------------+
| emp_id | emp_name | emp_phone | emp_email      |
+--------+----------+-----------+----------------+
|      3 | rama     | 67545     | ttrr@vcc.com   |
|      4 | bhima    | 65457     | gdfff@jhhf.com |
|      5 | shama    | 54464     | dffdff@gdd.com |
|      6 | rama     | 67545     | ttrr@vcc.com   |
|      7 | bhima    | 65457     | gdfff@jhhf.com |
|      8 | shama    | 54464     | dffdff@gdd.com |
+--------+----------+-----------+----------------+
6 rows in set (0.00 sec)


simulate data corruption by editing the os table file , if server restarted the table cannot be recreated
============================================================================================================

[root@mysqlserver employeesdb]# vi staff.ibd 
[root@mysqlserver employeesdb]# 
[root@mysqlserver employeesdb]# ls -rtl
total 112
-rw-r-----. 1 mysql mysql 114688 May 16 02:33 employees.ibd
-rw-r-----. 1 mysql mysql      0 May 16 02:36 staff.ibd
[root@mysqlserver employeesdb]# 

[root@mysqlserver employeesdb]# systemctl restart mysqld
[root@mysqlserver employeesdb]# 

[root@mysqlserver employeesdb]# mysqlcheck -u root -p -c employeesdb staff
Enter password: 

mysqlcheck: Got error: 2013: Lost connection to MySQL server during query when executing 'CHECK TABLE ... '

[root@mysqlserver employeesdb]# ls -rtl
total 112
-rw-r-----. 1 mysql mysql 114688 May 16 02:33 employees.ibd
-rw-r-----. 1 mysql mysql      0 May 16 02:36 staff.ibd
[root@mysqlserver employeesdb]# 

mysql> select * from staff;
ERROR 1812 (HY000): Tablespace is missing for table `employeesdb`.`staff`.
mysql> 

In [ ]:
mysqlshow program
================================

-- displays database , table and column information.

[mysqluser@mysqlserver ~]$ mysqlshow employeesdb

Database: employeesdb
+-----------+
|  Tables   |
+-----------+
| employees |
| staff     |
+-----------+


[mysqluser@mysqlserver ~]$ mysqlshow employeesdb employees

Database: employeesdb  Table: employees
+-----------+-------------+--------------------+------+-----+---------+----------------+---------------------------------+---------+
| Field     | Type        | Collation          | Null | Key | Default | Extra          | Privileges                      | Comment |
+-----------+-------------+--------------------+------+-----+---------+----------------+---------------------------------+---------+
| emp_id    | int         |                    | NO   | PRI |         | auto_increment | select,insert,update,references |         |
| emp_name  | varchar(50) | utf8mb4_0900_ai_ci | YES  |     |         |                | select,insert,update,references |         |
| emp_phone | varchar(50) | utf8mb4_0900_ai_ci | YES  |     |         |                | select,insert,update,references |         |
| emp_email | varchar(50) | utf8mb4_0900_ai_ci | YES  |     |         |                | select,insert,update,references |         |
+-----------+-------------+--------------------+------+-----+---------+----------------+---------------------------------+---------+


[mysqluser@mysqlserver ~]$ mysqlshow employeesdb employees emp_id

Database: employeesdb  Table: employees  Wildcard: emp_id
+--------+------+-----------+------+-----+---------+----------------+---------------------------------+---------+
| Field  | Type | Collation | Null | Key | Default | Extra          | Privileges                      | Comment |
+--------+------+-----------+------+-----+---------+----------------+---------------------------------+---------+
| emp_id | int  |           | NO   | PRI |         | auto_increment | select,insert,update,references |         |
+--------+------+-----------+------+-----+---------+----------------+---------------------------------+---------+
[mysqluser@mysqlserver ~]$ 


In [ ]:
mysql_tzinfo_to_sql program
=======================================

-- loads time zone data from zoneinfo database into mysql database.
-- location in linux is /usr/share/zoneinfo 


[mysqluser@mysqlserver ~]$ ls -rtl /usr/share/zoneinfo/
total 392
-rw-r--r--.  1 root root   4463 May 10  2021 iso3166.tab
-rw-r--r--.  1 root root   3388 Jul 22  2021 leapseconds
-rw-r--r--.  1 root root  19419 Sep 20  2021 zone.tab
-rw-r--r--.  1 root root  17593 Sep 25  2021 zone1970.tab
-rw-r--r--.  8 root root    127 Oct 26  2021 Zulu
-rw-r--r--.  2 root root   1544 Oct 26  2021 W-SU
-rw-r--r--.  1 root root   1873 Oct 26  2021 WET
-rw-r--r--.  8 root root    127 Oct 26  2021 UTC
-rw-r--r--.  8 root root    127 Oct 26  2021 Universal
-rw-r--r--.  8 root root    127 Oct 26  2021 UCT


mysql> use mysql

Database changed

mysql> show tables like 'time_zone%';

+------------------------------+
| Tables_in_mysql (time_zone%) |
+------------------------------+
| time_zone                    |
| time_zone_leap_second        |
| time_zone_name               |
| time_zone_transition         |
| time_zone_transition_type    |
+------------------------------+


mysql> select count(*) from time_zone;

+----------+
| count(*) |
+----------+
|        0 |
+----------+

mysql> select count(*) from time_zone_name;

+----------+
| count(*) |
+----------+
|        0 |
+----------+

[mysqluser@mysqlserver ~]$ mysql_tzinfo_to_sql /usr/share/zoneinfo/ | mysql mysql
Warning: Unable to load '/usr/share/zoneinfo//iso3166.tab' as time zone. Skipping it.
Warning: Unable to load '/usr/share/zoneinfo//leapseconds' as time zone. Skipping it.
Warning: Unable to load '/usr/share/zoneinfo//tzdata.zi' as time zone. Skipping it.
Warning: Unable to load '/usr/share/zoneinfo//zone.tab' as time zone. Skipping it.
Warning: Unable to load '/usr/share/zoneinfo//zone1970.tab' as time zone. Skipping it.
[mysqluser@mysqlserver ~]$ 


mysql> use mysql;

Database changed

mysql> select count(*) from time_zone;

+----------+
| count(*) |
+----------+
|     1783 |
+----------+

mysql> select count(*) from time_zone_name;

+----------+
| count(*) |
+----------+
|     1783 |
+----------+

In [ ]:
mysqlbinlog program 
=============================

-- binary logs or binlogs records changes in the database in binary format.
-- mysqlbinlog can be used to display the contents of this binary log or the relay log in replica.


output binlog content on screen
--------------------------------------

[root@bospprod01 mysql]# mysqlbinlog binlog.000006

# The proper term is pseudo_replica_mode, but we use this compatibility alias
# to make the statement usable on server versions 8.0.24 and older.
/*!50530 SET @@SESSION.PSEUDO_SLAVE_MODE=1*/;
/*!50003 SET @OLD_COMPLETION_TYPE=@@COMPLETION_TYPE,COMPLETION_TYPE=0*/;
DELIMITER /*!*/;
# at 4
#240528  3:41:48 server id 1  end_log_pos 126 CRC32 0x364cfcd2 	Start: binlog v 4, server v 8.0.30 
created 240528  
3:41:48
BINLOG '
vIpVZg8BAAAAegAAAH4AAAAAAAQAOC4wLjMwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAEwANAAgAAAAABAAEAAAAYgAEGggAAAAICAgCAAAACgoKKioAEjQA
CigAAdL8TDY=
'/*!*/;
# at 126
#240528  3:41:48 server id 1  end_log_pos 157 CRC32 0x0d45df00 	Previous-GTIDs


output binlog content to a file 
--------------------------------------

[root@bospprod01 mysql]# mysqlbinlog binlog.000006 > /tmp/binlog6.sql

[root@bospprod01 mysql]# ls -rtl /tmp/binlog6.sql 
-rw-r--r--. 1 root root 41629 May 31 05:18 /tmp/binlog6.sql
[root@bospprod01 mysql]# 


replace database name in the binlog to a different database name
-------------------------------------------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog --rewrite-db='airportdb->airports' binlog.000006 > /tmp/binlog6_airports.sql

[root@bospprod01 mysql]# cat /tmp/binlog6.sql | grep airport
#240528  3:50:05 server id 1  end_log_pos 399 CRC32 0x266013d7 	Table_map: `airportdb`.`passenger`
 mapped to number 85
#240528  3:53:29 server id 1  end_log_pos 4590 CRC32 0x2ae231a2 	Table_map: `airportdb`.`passenger`
 mapped to number 85

[root@bospprod01 mysql]# cat /tmp/binlog6_airports.sql | grep airport
#240528  3:50:05 server id 1  end_log_pos 399 CRC32 0x843296bc 	Table_map: `airports`.`passenger` 
mapped to number 85
#240528  3:53:29 server id 1  end_log_pos 4590 CRC32 0xb00bdb77 	Table_map: `airports`.`passenger`
 mapped to number 85
#240528  3:55:30 server id 1  end_log_pos 8781 CRC32 0xb89b7593 	Table_map: `airports`.`passenger` 
mapped to number 85


skip gtids inside the binlog 
-----------------------------------------

[root@bospprod01 mysql]# mysqlbinlog --skip-gtids binlog.000006 > /tmp/binlog6_skipgtid.sql
[root@bospprod01 mysql]# 


[root@bospprod01 mysql]# cat /tmp/binlog6.sql | grep -i gtid

#240528  3:41:48 server id 1  end_log_pos 157 CRC32 0x0d45df00 	Previous-GTIDs
#240528  3:50:05 server id 1  end_log_pos 236 CRC32 0x22e603e8 	Anonymous_GTID	last_committed=0	
sequence_number=1	rbr_only=yes	original_committed_timestamp=1716882606022148	
immediate_commit_timestamp=1716882606022148transaction_length=4191
SET @@SESSION.GTID_NEXT= 'ANONYMOUS'/*!*/;
#240528  3:53:29 server id 1  end_log_pos 4427 CRC32 0x0dbe241c 	Anonymous_GTID	last_committed=1	
sequence_number=2	rbr_only=yes	original_committed_timestamp=1716882810242724	
immediate_commit_timestamp=1716882810242724	transaction_length=4191
SET @@SESSION.GTID_NEXT= 'ANONYMOUS'/*!*/;

[root@bospprod01 mysql]# cat /tmp/binlog6_skipgtid.sql | grep -i gtid
/*!50616 SET @@SESSION.GTID_NEXT='AUTOMATIC'*//*!*/;
[root@bospprod01 mysql]# 


read binlog from a specicified date time 
---------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog --start-datetime="2024-05-28 03:56:00"  binlog.000006 > 
                         /tmp/binlog6_startdatetime.sql
[root@bospprod01 mysql]# 

[root@bospprod01 mysql]# ls -rtl /tmp/bin*
-rw-r--r--. 1 root root 41629 May 31 05:18 /tmp/binlog6.sql
-rw-r--r--. 1 root root 21645 May 31 07:16 /tmp/binlog6_startdatetime.sql
[root@bospprod01 mysql]# 


read binlog from a specified log position 
------------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog --start-position=14006  binlog.000006 > /tmp/binlog6_startpos.sql
[root@bospprod01 mysql]# 

[root@bospprod01 mysql]# ls -rtl /tmp/bin*
-rw-r--r--. 1 root root 41629 May 31 05:18 /tmp/binlog6.sql
-rw-r--r--. 1 root root 19000 May 31 07:22 /tmp/binlog6_startpos.sql
[root@bospprod01 mysql]# 


read binlog until a specified date time 
------------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog --stop-datetime="2024-05-28 03:56:00"  binlog.000006 > 
                         /tmp/binlog6_stopdatetime.sql
[root@bospprod01 mysql]# 

[root@bospprod01 mysql]# ls -rtl /tmp/bin*
-rw-r--r--. 1 root root 41629 May 31 05:18 /tmp/binlog6.sql
-rw-r--r--. 1 root root 21346 May 31 07:24 /tmp/binlog6_stopdatetime.sql
[root@bospprod01 mysql]# 


read binlog until a specified log position 
----------------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog --stop-position=14006  binlog.000006 > /tmp/binlog6_stoppos.sql
[root@bospprod01 mysql]# 
[root@bospprod01 mysql]# 
[root@bospprod01 mysql]# ls -rtl /tmp/bin*
-rw-r--r--. 1 root root 41629 May 31 05:18 /tmp/binlog6.sql
-rw-r--r--. 1 root root 24036 May 31 07:25 /tmp/binlog6_stoppos.sql
[root@bospprod01 mysql]# 


read binlog from a specified datetime until a specicified date time
---------------------------------------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog --start_datetime="2024-05-28 03:53:00" --stop-datetime="2024-05-28 03:57:00"  
                         binlog.000006 > /tmp/binlog6_startstopdatetime.sql
[root@bospprod01 mysql]# 
[root@bospprod01 mysql]# ls -rtl /tmp/bin*
-rw-r--r--. 1 root root 41629 May 31 05:18 /tmp/binlog6.sql
-rw-r--r--. 1 root root 23924 May 31 07:47 /tmp/binlog6_startstopdatetime.sql
[root@bospprod01 mysql]# 


read binlog from a specified log position until a specicified log position
---------------------------------------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog --start-position=4348 --stop-position=14006  binlog.000006 > 
                         /tmp/binlog6_startstoppos.sql
[root@bospprod01 mysql]# 
[root@bospprod01 mysql]# ls -rtl /tmp/bin*
-rw-r--r--. 1 root root 41629 May 31 05:18 /tmp/binlog6.sql
-rw-r--r--. 1 root root 17317 May 31 07:51 /tmp/binlog6_startstoppos.sql
[root@bospprod01 mysql]# 


read binlog from a specified log position until a specicified datetime
---------------------------------------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog --start-position=4348 --stop-datetime="2024-05-28 03:57:00"  binlog.000006 > 
                         /tmp/binlog6_startposstopdatetime.sql
[root@bospprod01 mysql]# 
[root@bospprod01 mysql]# ls -rtl /tmp/bin*
-rw-r--r--. 1 root root 41629 May 31 05:18 /tmp/binlog6.sql
-rw-r--r--. 1 root root 23926 May 31 07:54 /tmp/binlog6_startposstopdatetime.sql
[root@bospprod01 mysql]# 


read mutiple binlog files 
----------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog binlog.000006 binlog.000007 binlog.000008 > /tmp/binlog6_7_8.sql
[root@bospprod01 mysql]# 
[root@bospprod01 mysql]# mysqlbinlog binlog.00000[6-8] > /tmp/binlog_6_7_8.sql
[root@bospprod01 mysql]# 
[root@bospprod01 mysql]# ls -rtl /tmp/bin*
-rw-r--r--. 1 root root 46130 May 31 07:31 /tmp/binlog6_7_8.sql
-rw-r--r--. 1 root root 46130 May 31 07:32 /tmp/binlog_6_7_8.sql
[root@bospprod01 mysql]# 


by default row events are displayed as base64 encoded , to show it as sql use --verbose or -v 
------------------------------------------------------------------------------------------------

[root@bospprod01 mysql]# mysqlbinlog -v binlog.000006 > /tmp/binlog6_verbose.sql


[root@bospprod01 mysql]# ls -rtl /tmp/bin*
-rw-r--r--. 1 root root 41629 May 31 05:18 /tmp/binlog6.sql
-rw-r--r--. 1 root root 95081 May 31 08:14 /tmp/binlog6_verbose.sql
[root@bospprod01 mysql]# 


[root@bospprod01 mysql]# more /tmp/binlog6.sql

#240528  3:50:05 server id 1  end_log_pos 4317 CRC32 0xa257d7b2 	Update_rows: table id 85 flags: STMT_END_F
BINLOG '
rYxVZhMBAAAASAAAAI8BAAAAAFUAAAAAAAMACWFpcnBvcnRkYgAJcGFzc2VuZ2VyAAQD/g8PBv4k
kAGQAQABAQACAeDXE2Am
rYxVZh8BAAAATg8AAN0QAAAAAFUAAAAAAAEAAgAE//8AmwIAAAdQMTAzNjc3BABBbGFuBQBCYXRl



[root@bospprod01 mysql]# more /tmp/binlog6_verbose.sql

#240528  3:50:05 server id 1  end_log_pos 4317 CRC32 0xa257d7b2 	Update_rows: table id 85 flags: STMT_END_F
BINLOG '
rYxVZhMBAAAASAAAAI8BAAAAAFUAAAAAAAMACWFpcnBvcnRkYgAJcGFzc2VuZ2VyAAQD/g8PBv4k
kAGQAQABAQACAeDXE2Am
rYxVZh8BAAAATg8AAN0QAAAAAFUAAAAAAAEAAgAE//8AmwIAAAdQMTAzNjc3BABBbGFuBQBCYXRl

### UPDATE `airportdb`.`passenger`
### WHERE
###   @1=667
###   @2='P103677'
###   @3='Alan'
###   @4='Bates'
### SET
###   @1=667
###   @2='P103677'
###   @3='ALan'
###   @4='Bates'
### UPDATE `airportdb`.`passenger`
### WHERE
###   @1=684
###   @2='P103694'
###   @3='Alan'
###   @4='Arkin'


mysqlbinlog backup capabilities 
============================================

-- binary log can be used in crash recovery and point-in-time recovery.
-- mysqlbinlog can also read binary logs in binary format same as in text format.
-- so mysqlbinlog can be used to take backup of the binary logs to a different location.

-- the backup in binary format requires 2 options --raw and --read-from-remote-server


static backup 
--------------------


[root@bospprod01 backups]# mysqlbinlog --raw --read-from-remote-server binlog.000003 binlog.000004 binlog.000005
[root@bospprod01 backups]# 
[root@bospprod01 backups]# pwd
/home/mysqluser/backups
[root@bospprod01 backups]# ls -rtl
total 12
-rw-r-----. 1 root root  180 May 31 09:33 binlog.000003
-rw-r-----. 1 root root 1109 May 31 09:33 binlog.000004
-rw-r-----. 1 root root  720 May 31 09:33 binlog.000005
[root@bospprod01 backups]# 


backsup to the directory mentioned in the --result-file with bosp suffix 
----------------------------------------------------------------------------

[root@bospprod01 backups]# mysqlbinlog --raw --read-from-remote-server --result-file=/home/mysqluser/backups/bosp 
                           binlog.000003 binlog.000004 binlog.000005
[root@bospprod01 backups]# 

[root@bospprod01 backups]# ls -rtl /home/mysqluser/backups/bosp*
-rw-r-----. 1 root root  180 May 31 09:35 /home/mysqluser/backups/bospbinlog.000003
-rw-r-----. 1 root root 1109 May 31 09:35 /home/mysqluser/backups/bospbinlog.000004
-rw-r-----. 1 root root  720 May 31 09:35 /home/mysqluser/backups/bospbinlog.000005
[root@bospprod01 backups]# 


backsup till the last log file starting from the mentioned binlog file 
-------------------------------------------------------------------------

[root@bospprod01 backups]# mysqlbinlog --raw --read-from-remote-server --result-file=/home/mysqluser/backups/bosp 
                           --to-last-log binlog.000007
[root@bospprod01 backups]# 

[root@bospprod01 backups]# ls -rtl /home/mysqluser/backups/bosp*
-rw-r-----. 1 root root      909 May 31 09:38 /home/mysqluser/backups/bospbinlog.000008
-rw-r-----. 1 root root      180 May 31 09:38 /home/mysqluser/backups/bospbinlog.000007
-rw-r-----. 1 root root      180 May 31 09:38 /home/mysqluser/backups/bospbinlog.000009
-rw-r-----. 1 root root     2303 May 31 09:38 /home/mysqluser/backups/bospbinlog.000010
-rw-r-----. 1 root root 19625417 May 31 09:38 /home/mysqluser/backups/bospbinlog.000011
[root@bospprod01 backups]# 


Live backup 
-----------------------

[root@bospprod01 backups]# mysqlbinlog --raw --read-from-remote-server --result-file=/home/mysqluser/backups/bosp 
                           --stop-never binlog.000007 &
[1] 10447
[root@bospprod01 backups]# 

[root@bospprod01 backups]# jobs
[1]+  Running                 mysqlbinlog --raw --read-from-remote-server 
[root@bospprod01 backups]# 

[root@bospprod01 backups]# ls -rtl /home/mysqluser/backups/bosp*
-rw-r-----. 1 root root     2303 May 31 09:41 /home/mysqluser/backups/bospbinlog.000010
-rw-r-----. 1 root root      180 May 31 09:41 /home/mysqluser/backups/bospbinlog.000009
-rw-r-----. 1 root root      909 May 31 09:41 /home/mysqluser/backups/bospbinlog.000008
-rw-r-----. 1 root root      180 May 31 09:41 /home/mysqluser/backups/bospbinlog.000007
-rw-r-----. 1 root root 19625417 May 31 09:41 /home/mysqluser/backups/bospbinlog.000011
[root@bospprod01 backups]# 

mysql> flush logs;
Query OK, 0 rows affected (0.17 sec)

[root@bospprod01 backups]# ls -rtl /home/mysqluser/backups/bosp*
-rw-r-----. 1 root root     2303 May 31 09:41 /home/mysqluser/backups/bospbinlog.000010
-rw-r-----. 1 root root      180 May 31 09:41 /home/mysqluser/backups/bospbinlog.000009
-rw-r-----. 1 root root      909 May 31 09:41 /home/mysqluser/backups/bospbinlog.000008
-rw-r-----. 1 root root      180 May 31 09:41 /home/mysqluser/backups/bospbinlog.000007
-rw-r-----. 1 root root 19625461 May 31 09:43 /home/mysqluser/backups/bospbinlog.000011
-rw-r-----. 1 root root      157 May 31 09:43 /home/mysqluser/backups/bospbinlog.000012
[root@bospprod01 backups]# 

upon server stop the job will also be stopped 
--------------------------------------------------

[root@bospprod01 ~]# systemctl stop mysqld
[root@bospprod01 ~]# 
[root@bospprod01 ~]# jobs
[root@bospprod01 ~]# 


In [ ]:
mysqldump + mysqlbinlog for backup and restore 
=================================================================

Suppose we have mysqlbinlog backup running continuosly and we have a daily mysqldump running 
which takes a full database backup.

The below shows how to restore if any data loss occurs.

mysql> reset master;
Query OK, 0 rows affected (0.01 sec)

mysql> show binary logs;
+---------------------------+-----------+-----------+
| Log_name                  | File_size | Encrypted |
+---------------------------+-----------+-----------+
| mysqlprod02_binlog.000001 |       157 | No        |
+---------------------------+-----------+-----------+
1 row in set (0.00 sec)


[root@mysqlprod02 backups]# mysqlbinlog --read-from-remote-server --raw --stop-never 
                            --result-file=/home/mysqluser/backups/ mysqlprod02_binlog.000001 &
[1] 3442
[root@mysqlprod02 backups]# 
[root@mysqlprod02 backups]# jobs
[1]+  Running                 mysqlbinlog --read-from-remote-server --raw --stop-never --result-file=/home/mysqluser/backups/ mysqlprod02_binlog.000001 &
[root@mysqlprod02 backups]# 

[root@mysqlprod02 backups]# ls -rtl /home/mysqluser/backups/
total 4
-rw-r-----. 1 root root 157 Jun  1 01:56 mysqlprod02_binlog.000001
[root@mysqlprod02 backups]# 


mysql> call sp_insertrorderdetails_data(20001);
Query OK, 1 row affected (15.38 sec)



[root@mysqlprod02 ~]# mysqldump --all-databases --routines --events --master-data=2 > 
                                 /home/mysqluser/backups/fulldb_dump.sql 
WARNING: --master-data is deprecated and will be removed in a future version. Use --source-data instead.
[root@mysqlprod02 ~]# 

[root@mysqlprod02 ~]# ls -rtl /home/mysqluser/backups/fulldb_dump.sql
-rw-r--r--. 1 root root 7767161 Jun  1 02:01 /home/mysqluser/backups/fulldb_dump.sql
[root@mysqlprod02 ~]# 


mysql> call sp_insertrorderdetails_data(30001);
Query OK, 1 row affected (14.29 sec)

mysql> call sp_insertrorderdetails_data(40001);
Query OK, 1 row affected (12.53 sec)

mysql> flush logs;
Query OK, 0 rows affected (0.02 sec)

mysql> call sp_insertrorderdetails_data(50001);
Query OK, 1 row affected (11.07 sec)

mysql> flush logs;
Query OK, 0 rows affected (0.01 sec)

mysql> call sp_insertrorderdetails_data(60001);
Query OK, 1 row affected (11.08 sec)

mysql> call sp_insertrorderdetails_data(70001);
Query OK, 1 row affected (10.13 sec)


Simulate data loss by removing data files 
------------------------------------------------------------

[root@mysqlprod02 ~]# rm -rf /var/lib/mysql/ordersdb/*
[root@mysqlprod02 ~]# 

mysql> call sp_insertrorderdetails_data(80001);
Query OK, 1 row affected (12.95 sec)

mysql> call sp_insertrorderdetails_data(90001);
Query OK, 1 row affected (10.36 sec)

mysql> select count(*) from order_details;
+----------+
| count(*) |
+----------+
|   100000 |
+----------+
1 row in set (0.01 sec)

mysql> flush logs;
Query OK, 0 rows affected (0.02 sec)

mysql> flush tables;
Query OK, 0 rows affected (0.00 sec)


use mysqlcheck to analyse the tables 
--------------------------------------------------

[root@mysqlprod02 backups]# mysqlcheck -o ordersdb
ordersdb.order_details
note     : Table does not support optimize, doing recreate + analyze instead
error    : Got error 44 - 'InnoDB error' from storage engine
status   : Operation failed
ordersdb.order_item_details
note     : Table does not support optimize, doing recreate + analyze instead
error    : Got error 44 - 'InnoDB error' from storage engine
status   : Operation failed
[root@mysqlprod02 backups]# 


restore from the backup taken 
-----------------------------------------

[root@mysqlprod02 backups]# mysql < fulldb_dump.sql
[root@mysqlprod02 backups]# 


check the log file and position in the dump file 
---------------------------------------------------

-- CHANGE MASTER TO MASTER_LOG_FILE='mysqlprod02_binlog.000001', MASTER_LOG_POS=9425605;


restore the binlog files from this log position
---------------------------------------------------

[root@mysqlprod02 backups]# mysqlbinlog --start-position=9425605 mysqlprod02_binlog.000001 mysqlprod02_binlog.000002
                            mysqlprod02_binlog.000003 mysqlprod02_binlog.000004 | mysql


mysql> select count(*) from ordersdb.order_details;
+----------+
| count(*) |
+----------+
|    41618 |
+----------+
1 row in set (0.01 sec)

mysql> select min(order_id) from ordersdb.order_details;
+---------------+
| min(order_id) |
+---------------+
|             1 |
+---------------+
1 row in set (0.00 sec)

mysql> select max(order_id) from ordersdb.order_details;
+---------------+
| max(order_id) |
+---------------+
|         41618 |
+---------------+
1 row in set (0.00 sec)

mysql> flush logs;
Query OK, 0 rows affected (0.08 sec)

[root@mysqlprod02 backups]# mysqlbinlog mysqlprod02_binlog.000004 mysqlprod02_binlog.000005 | mysql
[root@mysqlprod02 backups]# 
